In [3]:
import pandas as pd
import re

In [4]:
twitter_data = pd.read_excel("./bpjs-20221109-123449.xlsx")
twitter_data.head()

,Unnamed: 0,tweet_id,text,favorite_count,retweet_count,created_at,source,reply_to_status,reply_to_user,retweets,favorites
0,0,1590216138642034944,@worksfess Berkasnya dipersulit buat persyarat...,0,0,2022-11-09 05:34:06,Twitter for Android,1.590185e+18,worksfess,0,0
1,1,1590215686944887040,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,0,0,2022-11-09 05:32:19,Twitter Web App,1.590213e+18,KacaLuthf1,0,0
2,2,1590215636286083072,RT @kafiradikalis: ...Mantep pelayanan klinik ...,0,1,2022-11-09 05:32:07,Twitter Web App,NaN,NaN,1,0
3,3,1590215293707514112,[titipan] di rsnd undip bisa buat konsul psiki...,1,0,2022-11-09 05:30:45,smngt kuliahnya dips! jgn gila,NaN,NaN,0,1
4,4,1590215214762689024,@18fessss Cepet periksa ke obgyn pk bpjs,0,0,2022-11-09 05:30:26,Twitter for iPhone,1.590014e+18,18fessss,0,0


In [5]:
df = pd.DataFrame(data=twitter_data['text'])
df.head()

,text
0,@worksfess Berkasnya dipersulit buat persyarat...
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...
3,[titipan] di rsnd undip bisa buat konsul psiki...
4,@18fessss Cepet periksa ke obgyn pk bpjs


Case Folding

In [6]:
#case folding
df['casefolding'] = df['text'].str.lower()
df

,text,casefolding
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs
...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,..."
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw..."


## Data Cleaning

In [7]:
def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', '', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', '', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', '', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', '', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', '', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', '', tweet)
    return tweet

def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word

def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', '', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', '', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        
        if is_valid_word(word):
            processed_tweet.append(word)

    return ' '.join(processed_tweet)

In [8]:
df['clean'] = df['casefolding'].apply(preprocess_tweet)

In [9]:
df.head()

,text,casefolding,clean
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs


## Tokenize

In [14]:
# NLTK word tokenize 
import nltk

def word_tokenize_wrapper(text):
    return nltk.word_tokenize(text)

df['tokenize'] = df['clean'].apply(word_tokenize_wrapper)
df

,text,casefolding,clean,stopwords_removal,tokenize
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti,"[ , , , , ]","[berkasnya, dipersulit, buat, persyaratan, bpj..."
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...,"[ , , , , , , , , 2, , , , , , ]","[udh, dicover, bpjs, aja, pasti, ada, pengelua..."
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...,"[ , , , , , , , , , , , , , , , ]","[mantep, pelayanan, klinik, eksekutif, di, rsh..."
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...,"[ , , , , , , , , , , , , , , , ...","[di, rsnd, undip, bisa, buat, konsul, psikiate..."
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs,"[ , , , , ]","[cepet, periksa, ke, obgyn, pk, bpjs]"
...,...,...,...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,...",gp mode on dok jangan kebanyakan rujuk nanti k...,"[ , , , , , , , , , , , , , , , ...","[gp, mode, on, dok, jangan, kebanyakan, rujuk,..."
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...,penyaluran bantuan langsung tunai blt akan dip...,"[ , , , , , , , , , , , , ]","[penyaluran, bantuan, langsung, tunai, blt, ak..."
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...,sudah tapi malah disuruh menghubungi dm cs kem...,"[ , , , , , , , , , , , , , , ]","[sudah, tapi, malah, disuruh, menghubungi, dm,..."
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw...",cw gaji minta pendapat work tapi gadapet bpjs ...,"[ , , , , , , , , , , , , ]","[cw, gaji, minta, pendapat, work, tapi, gadape..."


## Stopwords Removal

In [29]:
from nltk.corpus import stopwords

# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# stopword tambahan
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah','bak','haii', 'di'])

In [30]:
import pandas as pd
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)

In [31]:
#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['stopwords_removal'] = df['tokenize'].apply(stopwords_removal) 

df

,text,casefolding,clean,stopwords_removal,tokenize
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti,"[berkasnya, dipersulit, persyaratan, bpjs]","[berkasnya, dipersulit, buat, persyaratan, bpj..."
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...,"[udh, dicover, bpjs, pengeluaran, pribadi, lai...","[udh, dicover, bpjs, aja, pasti, ada, pengelua..."
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...,"[mantep, pelayanan, klinik, eksekutif, rshs, b...","[mantep, pelayanan, klinik, eksekutif, di, rsh..."
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...,"[rsnd, undip, konsul, psikiater, gapake, bpjs,...","[di, rsnd, undip, bisa, buat, konsul, psikiate..."
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs,"[cepet, periksa, obgyn, pk, bpjs]","[cepet, periksa, ke, obgyn, pk, bpjs]"
...,...,...,...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,...",gp mode on dok jangan kebanyakan rujuk nanti k...,"[gp, mode, dok, kebanyakan, rujuk, peringatan,...","[gp, mode, on, dok, jangan, kebanyakan, rujuk,..."
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...,penyaluran bantuan langsung tunai blt akan dip...,"[penyaluran, bantuan, tunai, blt, diperluas, w...","[penyaluran, bantuan, langsung, tunai, blt, ak..."
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...,sudah tapi malah disuruh menghubungi dm cs kem...,"[disuruh, menghubungi, dm, cs, kemnaker, nik]","[sudah, tapi, malah, disuruh, menghubungi, dm,..."
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw...",cw gaji minta pendapat work tapi gadapet bpjs ...,"[cw, gaji, pendapat, work, gadapet, bpjs, seje...","[cw, gaji, minta, pendapat, work, tapi, gadape..."


Norm

In [34]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['normalization'] = df['stopwords_removal'].apply(normalized_term)

df

,text,casefolding,clean,stopwords_removal,tokenize,normalization
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti,"[berkasnya, dipersulit, persyaratan, bpjs]","[berkasnya, dipersulit, buat, persyaratan, bpj...","[berkasnya, dipersulit, persyaratan, bpjs]"
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...,"[udh, dicover, bpjs, pengeluaran, pribadi, lai...","[udh, dicover, bpjs, aja, pasti, ada, pengelua...","[sudah, dicover, bpjs, pengeluaran, pribadi, l..."
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...,"[mantep, pelayanan, klinik, eksekutif, rshs, b...","[mantep, pelayanan, klinik, eksekutif, di, rsh...","[mantep, pelayanan, klinik, eksekutif, rshs, b..."
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...,"[rsnd, undip, konsul, psikiater, gapake, bpjs,...","[di, rsnd, undip, bisa, buat, konsul, psikiate...","[rsnd, undip, konsul, psikiater, gapake, bpjs,..."
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs,"[cepet, periksa, obgyn, pk, bpjs]","[cepet, periksa, ke, obgyn, pk, bpjs]","[cepet, periksa, obgyn, pk, bpjs]"
...,...,...,...,...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,...",gp mode on dok jangan kebanyakan rujuk nanti k...,"[gp, mode, dok, kebanyakan, rujuk, peringatan,...","[gp, mode, on, dok, jangan, kebanyakan, rujuk,...","[gp, mode, dok, kebanyakan, rujuk, peringatan,..."
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...,penyaluran bantuan langsung tunai blt akan dip...,"[penyaluran, bantuan, tunai, blt, diperluas, w...","[penyaluran, bantuan, langsung, tunai, blt, ak...","[penyaluran, bantuan, tunai, blt, diperluas, w..."
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...,sudah tapi malah disuruh menghubungi dm cs kem...,"[disuruh, menghubungi, dm, cs, kemnaker, nik]","[sudah, tapi, malah, disuruh, menghubungi, dm,...","[disuruh, menghubungi, dm, cs, kemnaker, nik]"
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw...",cw gaji minta pendapat work tapi gadapet bpjs ...,"[cw, gaji, pendapat, work, gadapet, bpjs, seje...","[cw, gaji, minta, pendapat, work, tapi, gadape...","[cw, gaji, pendapat, work, gadapet, bpjs, seje..."


## Stemming

In [49]:
#Memanggil library sastrawi 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['normalization']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    result =  [term_dict[term] for term in document]
    return ' '.join(result)

df['steeming'] = df['normalization'].swifter.apply(get_stemmed_term)
df

433
------------------------
berkasnya : berkas
dipersulit : sulit
persyaratan : syarat
bpjs : bpjs
sudah : sudah
dicover : dicover
pengeluaran : keluar
pribadi : pribadi
lain2 : lain2
perawatannya : awat
mantep : mantep
pelayanan : layan
klinik : klinik
eksekutif : eksekutif
rshs : rshs
bandung : bandung
tinggal : tinggal
bawa : bawa
uang : uang
sat : sat
set : set
ditangani : tangan
rsnd : rsnd
undip : undip
konsul : konsul
psikiater : psikiater
gapake : gapake
range : range
harga : harga
makasih : makasih
dips : dips
cepet : cepet
periksa : periksa
obgyn : obgyn
pk : pk
daerahku : daerah
gede : gede
segitu : segitu
ketenagakerjaan : ketenagakerjaan
daftar : daftar
bayar : bayar
mandiri : mandiri
penyaluran : salur
blt : blt
diperluas : luas
umkm : umkm
ojek : ojek
online : online
warga : warga
miskin : miskin
sebenernya : sebenernya
kesmas : kesmas
kesana : kesana
duit : duit
bantuan : bantu
tunai : tunai
pekerja : kerja
peserta : serta
lahir : lahir
normal : normal
mmg : mmg
gue : 

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

,text,casefolding,clean,stopwords_removal,tokenize,normalization,steeming,subjectivity
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti,"[berkasnya, dipersulit, persyaratan, bpjs]","[berkasnya, dipersulit, buat, persyaratan, bpj...","[berkasnya, dipersulit, persyaratan, bpjs]",berkas sulit syarat bpjs,0.0
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...,"[udh, dicover, bpjs, pengeluaran, pribadi, lai...","[udh, dicover, bpjs, aja, pasti, ada, pengelua...","[sudah, dicover, bpjs, pengeluaran, pribadi, l...",sudah dicover bpjs keluar pribadi lain2 awat,1.0
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...,"[mantep, pelayanan, klinik, eksekutif, rshs, b...","[mantep, pelayanan, klinik, eksekutif, di, rsh...","[mantep, pelayanan, klinik, eksekutif, rshs, b...",mantep layan klinik eksekutif rshs bandung tin...,0.0
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...,"[rsnd, undip, konsul, psikiater, gapake, bpjs,...","[di, rsnd, undip, bisa, buat, konsul, psikiate...","[rsnd, undip, konsul, psikiater, gapake, bpjs,...",rsnd undip konsul psikiater gapake bpjs range ...,0.0
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs,"[cepet, periksa, obgyn, pk, bpjs]","[cepet, periksa, ke, obgyn, pk, bpjs]","[cepet, periksa, obgyn, pk, bpjs]",cepet periksa obgyn pk bpjs,0.0
...,...,...,...,...,...,...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,...",gp mode on dok jangan kebanyakan rujuk nanti k...,"[gp, mode, dok, kebanyakan, rujuk, peringatan,...","[gp, mode, on, dok, jangan, kebanyakan, rujuk,...","[gp, mode, dok, kebanyakan, rujuk, peringatan,...",gp mode dok banyak rujuk ingat bpjs hhd,0.0
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...,penyaluran bantuan langsung tunai blt akan dip...,"[penyaluran, bantuan, tunai, blt, diperluas, w...","[penyaluran, bantuan, langsung, tunai, blt, ak...","[penyaluran, bantuan, tunai, blt, diperluas, w...",salur bantu tunai blt luas warga miskin kerja ...,0.0
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...,sudah tapi malah disuruh menghubungi dm cs kem...,"[disuruh, menghubungi, dm, cs, kemnaker, nik]","[sudah, tapi, malah, disuruh, menghubungi, dm,...","[disuruh, menghubungi, dm, cs, kemnaker, nik]",suruh hubung dm cs kemnaker nik,0.0
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw...",cw gaji minta pendapat work tapi gadapet bpjs ...,"[cw, gaji, pendapat, work, gadapet, bpjs, seje...","[cw, gaji, minta, pendapat, work, tapi, gadape...","[cw, gaji, pendapat, work, gadapet, bpjs, seje...",cw gaji dapat work gadapet bpjs jenis worth,0.1


,text,casefolding,clean,stopwords_removal,tokenize,normalization,steeming
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti,"[berkasnya, dipersulit, persyaratan, bpjs]","[berkasnya, dipersulit, buat, persyaratan, bpj...","[berkasnya, dipersulit, persyaratan, bpjs]","[berkas, sulit, syarat, bpjs]"
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...,"[udh, dicover, bpjs, pengeluaran, pribadi, lai...","[udh, dicover, bpjs, aja, pasti, ada, pengelua...","[sudah, dicover, bpjs, pengeluaran, pribadi, l...","[sudah, dicover, bpjs, keluar, pribadi, lain2,..."
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...,"[mantep, pelayanan, klinik, eksekutif, rshs, b...","[mantep, pelayanan, klinik, eksekutif, di, rsh...","[mantep, pelayanan, klinik, eksekutif, rshs, b...","[mantep, layan, klinik, eksekutif, rshs, bandu..."
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...,"[rsnd, undip, konsul, psikiater, gapake, bpjs,...","[di, rsnd, undip, bisa, buat, konsul, psikiate...","[rsnd, undip, konsul, psikiater, gapake, bpjs,...","[rsnd, undip, konsul, psikiater, gapake, bpjs,..."
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs,"[cepet, periksa, obgyn, pk, bpjs]","[cepet, periksa, ke, obgyn, pk, bpjs]","[cepet, periksa, obgyn, pk, bpjs]","[cepet, periksa, obgyn, pk, bpjs]"
...,...,...,...,...,...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,...",gp mode on dok jangan kebanyakan rujuk nanti k...,"[gp, mode, dok, kebanyakan, rujuk, peringatan,...","[gp, mode, on, dok, jangan, kebanyakan, rujuk,...","[gp, mode, dok, kebanyakan, rujuk, peringatan,...","[gp, mode, dok, banyak, rujuk, ingat, bpjs, hhd]"
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...,penyaluran bantuan langsung tunai blt akan dip...,"[penyaluran, bantuan, tunai, blt, diperluas, w...","[penyaluran, bantuan, langsung, tunai, blt, ak...","[penyaluran, bantuan, tunai, blt, diperluas, w...","[salur, bantu, tunai, blt, luas, warga, miskin..."
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...,sudah tapi malah disuruh menghubungi dm cs kem...,"[disuruh, menghubungi, dm, cs, kemnaker, nik]","[sudah, tapi, malah, disuruh, menghubungi, dm,...","[disuruh, menghubungi, dm, cs, kemnaker, nik]","[suruh, hubung, dm, cs, kemnaker, nik]"
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw...",cw gaji minta pendapat work tapi gadapet bpjs ...,"[cw, gaji, pendapat, work, gadapet, bpjs, seje...","[cw, gaji, minta, pendapat, work, tapi, gadape...","[cw, gaji, pendapat, work, gadapet, bpjs, seje...","[cw, gaji, dapat, work, gadapet, bpjs, jenis, ..."


## Drop Duplikat

In [96]:
for i, text in enumerate(df['steeming']):
    if (len(text) == 0):
        df = df.drop(i)

In [97]:
for i, text in enumerate(df['steeming']):
    if (len(text) == 0):
         print('ok')

## Labeling

In [61]:
from textblob import TextBlob
def to_subjectivity(sentence):
    return TextBlob(sentence).sentiment.subjectivity

def to_polarity(sentence):
    a = TextBlob(sentence)
    translated = a.translate(from_lang="id", to="en")
    return translated.sentiment.polarity

In [98]:
df['subjectivity'] = df['steeming'].apply(to_subjectivity)
df['polarity'] = df['steeming'].apply(to_polarity)
df

,text,casefolding,clean,stopwords_removal,tokenize,normalization,steeming,subjectivity,polarity
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti,"[berkasnya, dipersulit, persyaratan, bpjs]","[berkasnya, dipersulit, buat, persyaratan, bpj...","[berkasnya, dipersulit, persyaratan, bpjs]",berkas sulit syarat bpjs,0.0,-0.500000
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...,"[udh, dicover, bpjs, pengeluaran, pribadi, lai...","[udh, dicover, bpjs, aja, pasti, ada, pengelua...","[sudah, dicover, bpjs, pengeluaran, pribadi, l...",sudah dicover bpjs keluar pribadi lain2 awat,0.0,-0.125000
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...,"[mantep, pelayanan, klinik, eksekutif, rshs, b...","[mantep, pelayanan, klinik, eksekutif, di, rsh...","[mantep, pelayanan, klinik, eksekutif, rshs, b...",mantep layan klinik eksekutif rshs bandung tin...,0.0,0.000000
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...,"[rsnd, undip, konsul, psikiater, gapake, bpjs,...","[di, rsnd, undip, bisa, buat, konsul, psikiate...","[rsnd, undip, konsul, psikiater, gapake, bpjs,...",rsnd undip konsul psikiater gapake bpjs range ...,0.0,0.200000
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs,"[cepet, periksa, obgyn, pk, bpjs]","[cepet, periksa, ke, obgyn, pk, bpjs]","[cepet, periksa, obgyn, pk, bpjs]",cepet periksa obgyn pk bpjs,0.0,0.333333
...,...,...,...,...,...,...,...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,...",gp mode on dok jangan kebanyakan rujuk nanti k...,"[gp, mode, dok, kebanyakan, rujuk, peringatan,...","[gp, mode, on, dok, jangan, kebanyakan, rujuk,...","[gp, mode, dok, kebanyakan, rujuk, peringatan,...",gp mode dok banyak rujuk ingat bpjs hhd,0.0,0.500000
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...,penyaluran bantuan langsung tunai blt akan dip...,"[penyaluran, bantuan, tunai, blt, diperluas, w...","[penyaluran, bantuan, langsung, tunai, blt, ak...","[penyaluran, bantuan, tunai, blt, diperluas, w...",salur bantu tunai blt luas warga miskin kerja ...,0.0,0.000000
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...,sudah tapi malah disuruh menghubungi dm cs kem...,"[disuruh, menghubungi, dm, cs, kemnaker, nik]","[sudah, tapi, malah, disuruh, menghubungi, dm,...","[disuruh, menghubungi, dm, cs, kemnaker, nik]",suruh hubung dm cs kemnaker nik,0.0,0.000000
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw...",cw gaji minta pendapat work tapi gadapet bpjs ...,"[cw, gaji, pendapat, work, gadapet, bpjs, seje...","[cw, gaji, minta, pendapat, work, tapi, gadape...","[cw, gaji, pendapat, work, gadapet, bpjs, seje...",cw gaji dapat work gadapet bpjs jenis worth,0.1,0.300000


In [100]:
def labeling(score):
    if score < 0:
        return 'negative'
    elif score == 0:
        return 'neutral'
    else:
        return 'positive'
    
df['label'] = df['polarity'].apply(labeling)
df

,text,casefolding,clean,stopwords_removal,tokenize,normalization,steeming,subjectivity,polarity,label
0,@worksfess Berkasnya dipersulit buat persyarat...,@worksfess berkasnya dipersulit buat persyarat...,berkasnya dipersulit buat persyaratan bpjs nanti,"[berkasnya, dipersulit, persyaratan, bpjs]","[berkasnya, dipersulit, buat, persyaratan, bpj...","[berkasnya, dipersulit, persyaratan, bpjs]",berkas sulit syarat bpjs,0.0,-0.500000,negative
1,@KacaLuthf1 udh dicover bpjs aja pasti ada pen...,@kacaluthf1 udh dicover bpjs aja pasti ada pen...,udh dicover bpjs aja pasti ada pengeluaran pri...,"[udh, dicover, bpjs, pengeluaran, pribadi, lai...","[udh, dicover, bpjs, aja, pasti, ada, pengelua...","[sudah, dicover, bpjs, pengeluaran, pribadi, l...",sudah dicover bpjs keluar pribadi lain2 awat,0.0,-0.125000,negative
2,RT @kafiradikalis: ...Mantep pelayanan klinik ...,rt @kafiradikalis: ...mantep pelayanan klinik ...,mantep pelayanan klinik eksekutif di rshs band...,"[mantep, pelayanan, klinik, eksekutif, rshs, b...","[mantep, pelayanan, klinik, eksekutif, di, rsh...","[mantep, pelayanan, klinik, eksekutif, rshs, b...",mantep layan klinik eksekutif rshs bandung tin...,0.0,0.000000,neutral
3,[titipan] di rsnd undip bisa buat konsul psiki...,[titipan] di rsnd undip bisa buat konsul psiki...,di rsnd undip bisa buat konsul psikiater ga se...,"[rsnd, undip, konsul, psikiater, gapake, bpjs,...","[di, rsnd, undip, bisa, buat, konsul, psikiate...","[rsnd, undip, konsul, psikiater, gapake, bpjs,...",rsnd undip konsul psikiater gapake bpjs range ...,0.0,0.200000,positive
4,@18fessss Cepet periksa ke obgyn pk bpjs,@18fessss cepet periksa ke obgyn pk bpjs,cepet periksa ke obgyn pk bpjs,"[cepet, periksa, obgyn, pk, bpjs]","[cepet, periksa, ke, obgyn, pk, bpjs]","[cepet, periksa, obgyn, pk, bpjs]",cepet periksa obgyn pk bpjs,0.0,0.333333,positive
...,...,...,...,...,...,...,...,...,...,...
95,"RT @dhL1453: @FebyPurnama GP mode on:\n\n""Dok,...","rt @dhl1453: @febypurnama gp mode on:\n\n""dok,...",gp mode on dok jangan kebanyakan rujuk nanti k...,"[gp, mode, dok, kebanyakan, rujuk, peringatan,...","[gp, mode, on, dok, jangan, kebanyakan, rujuk,...","[gp, mode, dok, kebanyakan, rujuk, peringatan,...",gp mode dok banyak rujuk ingat bpjs hhd,0.0,0.500000,positive
96,RT @garuda080: Penyaluran bantuan langsung tun...,rt @garuda080: penyaluran bantuan langsung tun...,penyaluran bantuan langsung tunai blt akan dip...,"[penyaluran, bantuan, tunai, blt, diperluas, w...","[penyaluran, bantuan, langsung, tunai, blt, ak...","[penyaluran, bantuan, tunai, blt, diperluas, w...",salur bantu tunai blt luas warga miskin kerja ...,0.0,0.000000,neutral
97,@PosIndonesia Sudah. Tapi malah disuruh menghu...,@posindonesia sudah. tapi malah disuruh menghu...,sudah tapi malah disuruh menghubungi dm cs kem...,"[disuruh, menghubungi, dm, cs, kemnaker, nik]","[sudah, tapi, malah, disuruh, menghubungi, dm,...","[disuruh, menghubungi, dm, cs, kemnaker, nik]",suruh hubung dm cs kemnaker nik,0.0,0.000000,neutral
98,"RT @worksfess: CW // Gaji, Minta Pendapat\n\nW...","rt @worksfess: cw // gaji, minta pendapat\n\nw...",cw gaji minta pendapat work tapi gadapet bpjs ...,"[cw, gaji, pendapat, work, gadapet, bpjs, seje...","[cw, gaji, minta, pendapat, work, tapi, gadape...","[cw, gaji, pendapat, work, gadapet, bpjs, seje...",cw gaji dapat work gadapet bpjs jenis worth,0.1,0.300000,positive
